# Modelo de prediccion linear

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Cargar el archivo CSV
df = pd.read_csv('/content/Libro1.csv', delimiter=';')

# Convertir la columna 'numeros' en una lista de listas
numeros = df['Numero'].str.split('-').apply(lambda x: [int(i) for i in x])

# Convertir en un único array 1D
data = np.concatenate(numeros.values)
def predict_next_seven_numbers(model, last_seven):
    predictions = []

    for _ in range(7):
        X_next = np.array([[len(last_seven) + len(predictions)]])
        next_pred = model.predict(X_next)[0]

        # Redondear y convertir en entero
        next_pred_rounded = int(round(next_pred))

        predictions.append(next_pred_rounded)
        last_seven = np.append(last_seven[1:], next_pred_rounded)

    return predictions


# Entrena el modelo con todos los datos
X = np.arange(len(data)).reshape(-1, 1)
model = LinearRegression().fit(X, data)

# Asume que deseas predecir los próximos 7 números basándote en los últimos 7 números de tu dataset
last_seven_numbers = data[-7:]
predictions = predict_next_seven_numbers(model, last_seven_numbers)

print(f"Los próximos 7 números predichos son: {predictions}")


Los próximos 7 números predichos son: [8, 9, 10, 11, 12, 13, 14]


In [ ]:
print(df['Fecha'])
print(df['Numero'])


# Modelo usando tecna de LSMT para prediccion de datos


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

# Cargar y preparar los datos
df = pd.read_csv('/content/Libro2.csv', delimiter=';')
numeros = df['Numero'].str.split('-').apply(lambda x: [int(i) for i in x])
data = np.concatenate(numeros.values).reshape(-1, 1)

# Reescalado de los datos
scaler = MinMaxScaler(feature_range=(0, 1))
data_normalized = scaler.fit_transform(data)

# Preparar los datos para LSTM
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 7
X, y = create_dataset(data_normalized, look_back)
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))

# Construir y entrenar modelo LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X, y, epochs=100, batch_size=1)

# ... (resto del código)

# Predicciones
predictions = []
input_data = data_normalized[-look_back:].reshape(1, 1, look_back)

for _ in range(7):
    pred_normalized = model.predict(input_data)
    pred_value = int(round(scaler.inverse_transform(pred_normalized).flatten()[0]))
    predictions.append(pred_value)

    # Desplaza la ventana de entrada
    input_data = np.roll(input_data, -1)
    input_data[0, 0, -1] = scaler.transform([[pred_value]])

print(f"Los próximos 7 números predichos son: {predictions}")




# Modelo usando tecna de LSMT para prediccion de datos Usando dropout y early stop

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping

# Cargar y preparar los datos
df = pd.read_csv('/content/Libro4.csv', delimiter=';')
numeros = df['Numero'].str.split('-').apply(lambda x: [int(i) for i in x])
data = np.concatenate(numeros.values).reshape(-1, 1)

# Reescalado de los datos
scaler = MinMaxScaler(feature_range=(0, 1))
data_normalized = scaler.fit_transform(data)

# Preparar los datos para LSTM
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 7
X, y = create_dataset(data_normalized, look_back)
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))

# Construir y entrenar modelo LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(1, look_back)))
model.add(Dropout(0.2))  # Capa Dropout después de la primera capa LSTM
model.add(LSTM(50))
model.add(Dropout(0.2))  # Capa Dropout después de la segunda capa LSTM
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Aplicar EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
model.fit(X, y, epochs=50, batch_size=1, callbacks=[early_stopping])

# Predicciones
predictions = []
input_data = data_normalized[-look_back:].reshape(1, 1, look_back)
predicted_set = set()

for _ in range(7):
    pred_normalized = model.predict(input_data)
    pred_value = int(round(scaler.inverse_transform(pred_normalized).flatten()[0]))

    # Asegurar que no haya números repetidos
    while pred_value in predicted_set:
        pred_value += 1
    predicted_set.add(pred_value)

    predictions.append(pred_value)

    # Desplaza la ventana de entrada
    input_data = np.roll(input_data, -1)
    input_data[0, 0, -1] = scaler.transform([[pred_value]])

predictions = sorted(predictions)  # Ordenar las predicciones
print(f"Los próximos 7 números predichos son: {predictions}")



Epoch 1/50
70/70 [==============================] - 9s 3ms/step - loss: 0.1487
Epoch 2/50
70/70 [==============================] - 0s 3ms/step - loss: 0.0730
Epoch 3/50
70/70 [==============================] - 0s 3ms/step - loss: 0.0685
Epoch 4/50
70/70 [==============================] - 0s 3ms/step - loss: 0.0643
Epoch 5/50
70/70 [==============================] - 0s 3ms/step - loss: 0.0546
Epoch 6/50
70/70 [==============================] - 0s 3ms/step - loss: 0.0494
Epoch 7/50
70/70 [==============================] - 0s 3ms/step - loss: 0.0533
Epoch 8/50
70/70 [==============================] - 0s 3ms/step - loss: 0.0498
Epoch 9/50
70/70 [==============================] - 0s 3ms/step - loss: 0.0460
Epoch 10/50
70/70 [==============================] - 0s 3ms/step - loss: 0.0505
Epoch 11/50
70/70 [==============================] - 0s 5ms/step - loss: 0.0501
Epoch 12/50
70/70 [==============================] - 0s 5ms/step - loss: 0.0509
Epoch 13/50
70/70 [==============================

# Guardar el Modelo Existente




In [ ]:
model.save('mi_modelo_lstm.h5')


# Cargar el Modelo Existente


In [ ]:
from keras.models import load_model

model = load_model('mi_modelo_lstm.h5')


# Entrenar con Nuevos Datos


In [ ]:
# Suponiendo que 'nuevos_datos.csv' es tu archivo con nuevos datos
df_nuevo = pd.read_csv('/content/Libro1.csv', delimiter=';')
numeros_nuevo = df_nuevo['Numero'].str.split('-').apply(lambda x: [int(i) for i in x])
data_nuevo = np.concatenate(numeros_nuevo.values).reshape(-1, 1)
data_normalized_nuevo = scaler.transform(data_nuevo)  # Usa el mismo scaler
X_nuevo, y_nuevo = create_dataset(data_normalized_nuevo, look_back)
X_nuevo = np.reshape(X_nuevo, (X_nuevo.shape[0], 1, X_nuevo.shape[1]))

# Entrenamiento con los nuevos datos
model.fit(X_nuevo, y_nuevo, epochs=100, batch_size=1, callbacks=[early_stopping])

# ... (Luego de entrenar con los nuevos datos)

# Predicciones
predictions = []
input_data = data_normalized_nuevo[-look_back:].reshape(1, 1, look_back)

for _ in range(7):
    pred_normalized = model.predict(input_data)
    pred_value = int(round(scaler.inverse_transform(pred_normalized).flatten()[0]))
    predictions.append(pred_value)

    # Desplaza la ventana de entrada
    input_data = np.roll(input_data, -1)
    input_data[0, 0, -1] = scaler.transform([[pred_value]])

print(f"Los próximos 7 números predichos son: {predictions}")



# Modelo usando tecna de LSMT para prediccion de datos Usando dropout y early stop con 6 numeros

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping

# Cargar y preparar los datos
df = pd.read_csv('/content/resultadosV2.csv', delimiter=';')
numeros = df['Numero'].str.split('-').apply(lambda x: [int(i) for i in x])
data = np.concatenate(numeros.values).reshape(-1, 1)

# Reescalado de los datos
scaler = MinMaxScaler(feature_range=(0, 1))
data_normalized = scaler.fit_transform(data)

# Preparar los datos para LSTM
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 7
X, y = create_dataset(data_normalized, look_back)
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))

# Construir y entrenar modelo LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(1, look_back)))
model.add(Dropout(0.2))  # Capa Dropout después de la primera capa LSTM
model.add(LSTM(50))
model.add(Dropout(0.2))  # Capa Dropout después de la segunda capa LSTM
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Aplicar EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
model.fit(X, y, epochs=10, batch_size=1, callbacks=[early_stopping])

# Predicciones
predictions = []
input_data = data_normalized[-look_back:].reshape(1, 1, look_back)
predicted_set = set()

for _ in range(6):
    pred_normalized = model.predict(input_data)
    pred_value = int(round(scaler.inverse_transform(pred_normalized).flatten()[0]))

    # Asegurar que no haya números repetidos
    while pred_value in predicted_set:
        pred_value += 1
    predicted_set.add(pred_value)

    predictions.append(pred_value)

    # Desplaza la ventana de entrada
    input_data = np.roll(input_data, -1)
    input_data[0, 0, -1] = scaler.transform([[pred_value]])

predictions = sorted(predictions)  # Ordenar las predicciones
print(f"Los próximos 6 números predichos son: {predictions}")


Epoch 1/10
19751/19751 [==============================] - 60s 3ms/step - loss: 0.0239
Epoch 2/10
19751/19751 [==============================] - 56s 3ms/step - loss: 0.0156
Epoch 3/10
19751/19751 [==============================] - 56s 3ms/step - loss: 0.0142
Epoch 4/10
19751/19751 [==============================] - 56s 3ms/step - loss: 0.0139
Epoch 5/10
19751/19751 [==============================] - 58s 3ms/step - loss: 0.0136
Epoch 6/10
19751/19751 [==============================] - 56s 3ms/step - loss: 0.0136
Epoch 7/10
19751/19751 [==============================] - 57s 3ms/step - loss: 0.0135
Epoch 8/10
19751/19751 [==============================] - 56s 3ms/step - loss: 0.0135
Epoch 9/10
19751/19751 [==============================] - 56s 3ms/step - loss: 0.0134
Epoch 10/10
19751/19751 [==============================] - 58s 3ms/step - loss: 0.0134


# Modelo usando tecna de LSMT para prediccion de datos Usando dropout y early stop con 1 numero

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping

# Cargar y preparar los datos
df = pd.read_csv('/content/resultados_1number.csv', delimiter=';')
data = df['Numero'].values.reshape(-1, 1)  # directly reshape the data


# Reescalado de los datos
scaler = MinMaxScaler(feature_range=(0, 1))
data_normalized = scaler.fit_transform(data)

# Preparar los datos para LSTM
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 7
X, y = create_dataset(data_normalized, look_back)
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))

# Construir y entrenar modelo LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(1, look_back)))
model.add(Dropout(0.2))  # Capa Dropout después de la primera capa LSTM
model.add(LSTM(50))
model.add(Dropout(0.2))  # Capa Dropout después de la segunda capa LSTM
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Aplicar EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
model.fit(X, y, epochs=200, batch_size=1, callbacks=[early_stopping])

# Predicciones
predictions = []
input_data = data_normalized[-look_back:].reshape(1, 1, look_back)
predicted_set = set()

for _ in range(1):
    pred_normalized = model.predict(input_data)
    pred_value = int(round(scaler.inverse_transform(pred_normalized).flatten()[0]))

    # Asegurar que no haya números repetidos
    while pred_value in predicted_set:
        pred_value += 1
    predicted_set.add(pred_value)

    predictions.append(pred_value)

    # Desplaza la ventana de entrada
    input_data = np.roll(input_data, -1)
    input_data[0, 0, -1] = scaler.transform([[pred_value]])

predictions = sorted(predictions)  # Ordenar las predicciones
print(f"El próximo número predicho es: {predictions[0]}")


Epoch 1/200
329/329 [==============================] - 4s 3ms/step - loss: 0.1159
Epoch 2/200
329/329 [==============================] - 1s 3ms/step - loss: 0.1010
Epoch 3/200
329/329 [==============================] - 1s 3ms/step - loss: 0.0993
Epoch 4/200
329/329 [==============================] - 1s 3ms/step - loss: 0.0972
Epoch 5/200
329/329 [==============================] - 1s 3ms/step - loss: 0.1024
Epoch 6/200
329/329 [==============================] - 1s 3ms/step - loss: 0.0982
Epoch 7/200
329/329 [==============================] - 1s 3ms/step - loss: 0.0971
Epoch 8/200
329/329 [==============================] - 1s 4ms/step - loss: 0.0985
Epoch 9/200
329/329 [==============================] - 1s 4ms/step - loss: 0.1003
Epoch 10/200
329/329 [==============================] - 1s 3ms/step - loss: 0.0968
Epoch 11/200
329/329 [==============================] - 1s 3ms/step - loss: 0.0973
Epoch 12/200
329/329 [==============================] - 1s 3ms/step - loss: 0.0972
Epoch 13/200
